<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RunnableLamda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. `RunnableLambda`**
---
`RunnableLambda` es un componente clave en LangChain Expression Language (LCEL) que permite
integrar funciones de Python estándar directamente en tus cadenas y flujos de trabajo.
En esencia, `RunnableLambda` toma una función de Python y la convierte en un objeto
"runnable" que puede ser parte de una cadena de LangChain, recibiendo la entrada del
paso anterior y pasando su salida al siguiente.

**¿Por qué usar RunnableLambda?**

* **Flexibilidad:** Integra lógica de Python arbitraria dentro de tus cadenas de LangChain.  

* **Simplicidad:** Envuelve funciones existentes sin necesidad de crear clases complejas.
* **Modularidad:** Permite descomponer tareas complejas en funciones más pequeñas y reutilizables.
* **Integración:** Facilita la conexión de LangChain con herramientas y librerías de Python.

Vamos a ver a continuación varios ejemplos de uso

# **Preparando el entorno del cuaderno**
---
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together y Anthropic

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.
El uso de las API de OpenAI y Anthropic es de pago. El resto son gratuitas y para usarlas basta con registrase y generar una API Key.  

En el primer cuaderno encontraras los enlaces a estos servicios y este codigo explicado

In [1]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

## **Ejemplo 1: Envolviendo funciones simples**
---

La forma principal de añadir una función a una cadena en Langchain (especialmente en LCEL) es utilizando RunnableLambda. RunnableLambda es un Runnable que envuelve una función Python, permitiéndole integrarse perfectamente en el flujo de la cadena.

(Aunque es posible, la inserción directa de una función en una cadena funciona, esto se debe a que Langchain implícitamente envuelve esa función en un RunnableLambda "por debajo del capó" para que pueda encajar dentro del paradigma de la cadena LCEL. Es mejor ser consistente con los principios del framework y usar RunnableLambda)

In [3]:
from langchain_core.runnables import RunnableLambda

# Función simple de Python
def duplicar(x):
    # Duplica el valor de entrada
    return x * 2

# Convertir a Runnable
runnable_duplicar = RunnableLambda(duplicar)

# Invocar el Runnable
resultado = runnable_duplicar.invoke(5)
print(f"Resultado de duplicar: {resultado}")  # Salida: 10

Resultado de duplicar: 10


## Ejemplo 2: Usando RunnableLambda en una cadena simple

Múltiples `RunnableLambda` en secuencia. Este ejemplo muestra cómo encadenar múltiples `RunnableLambda`. La salida del primero (`runnable_mayusculas`) se convierte en la entrada del segundo
(`runnable_exclamacion`).

In [5]:
def mayusculas(texto):
    return texto.upper()

def agregar_exclamacion(texto):
    return texto + "!"

runnable_mayusculas = RunnableLambda(mayusculas)
runnable_exclamacion = RunnableLambda(agregar_exclamacion)

cadena_transformacion = runnable_mayusculas | runnable_exclamacion

resultado_transformacion = cadena_transformacion.invoke("Este es un texto cualquiera")
print(resultado_transformacion)

ESTE ES UN TEXTO CUALQUIERA!
